In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('Microsoft_Data.csv')
data.shape
data.info
data.isnull().sum().sort_values(ascending=False)#Columns which are having more nan values splitted
data.drop(columns=['PuaMode','Census_ProcessorClass','DefaultBrowsersIdentifier','Census_IsFlightingInternal',
                   'Census_InternalBatteryType','Census_ThresholdOptIn','Census_IsWIMBootEnabled'],inplace=True)
data.shape

(56756, 76)

In [2]:
data['MachineIdentifier'].unique()

array(['0000028988387b115f69f31a3bf04f09',
       '000007535c3f730efa9ea0b7ef1bd645',
       '000007905a28d863f6d0d597892cd692', ...,
       '01a3af52c05b6af7d971d5f3092c3a80',
       '01a3b0c12114367eb99eaa2fac6b754c',
       '01a3b14d2c2d4f1bc85204d10713884d'], dtype=object)

In [3]:
data_obj=data.select_dtypes('object')

In [4]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
data_num=data.select_dtypes(numerics)

In [5]:
#print(len(data['IsBeta'].unique()))
for col in data_num :
    unique_values=len(data_num[col].unique())
    print(col,unique_values>10000)


IsBeta False
RtpStateBitfield False
IsSxsPassiveMode False
AVProductStatesIdentifier False
AVProductsInstalled False
AVProductsEnabled False
HasTpm False
CountryIdentifier False
CityIdentifier True
OrganizationIdentifier False
GeoNameIdentifier False
LocaleEnglishNameIdentifier False
OsBuild False
OsSuite False
IsProtected False
AutoSampleOptIn False
SMode False
IeVerIdentifier False
Firewall False
UacLuaenable False
Census_OEMNameIdentifier False
Census_OEMModelIdentifier True
Census_ProcessorCoreCount False
Census_ProcessorManufacturerIdentifier False
Census_ProcessorModelIdentifier False
Census_PrimaryDiskTotalCapacity False
Census_SystemVolumeTotalCapacity True
Census_HasOpticalDiskDrive False
Census_TotalPhysicalRAM False
Census_InternalPrimaryDiagonalDisplaySizeInInches False
Census_InternalPrimaryDisplayResolutionHorizontal False
Census_InternalPrimaryDisplayResolutionVertical False
Census_InternalBatteryNumberOfCharges False
Census_OSBuildNumber False
Census_OSBuildRevision Fal

In [6]:
#Converting all values to categorical

for col in data_num :
    data_num[col]=data_num[col].astype('object')




C:\Users\Ramya\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
#data_object=data_obj+data_num
data_object=pd.concat([data_num,data_obj],axis=1)
data_object.shape
y=data_object['HasDetections']
data_object.drop(columns='HasDetections',inplace=True)


In [8]:
columns=list(data_object.columns)

for col in data_object:
    
    # remove columns with high NA rate
    na_rate = data_object[col].isnull().sum() / data_object.shape[0]
    
    # remove columns with high Unbalanced values rate
    unbalanced_rate = data_object[col].value_counts(normalize=True, dropna=False).values[0]
    
    if na_rate > 0.7:
        columns.remove(col)
    elif unbalanced_rate > 0.9:
        columns.remove(col)

In [9]:
columns
data_object=data_object[columns]
data_object.shape

(56756, 52)

In [10]:
for col in data_object:
    data_object[col]=data_object[col].fillna(data_object[col].mode().iloc[0])
data_series=list(data_object.columns)

In [11]:
import scipy
from scipy import stats

col_p_values=[]

p_values_list=[]
for col in data_object:
    crosstab=pd.crosstab(data_object[col],y)# build Contigency table
    data_object[col].value_counts()
    p_values_list.append(scipy.stats.chi2_contingency(crosstab)[1])
    
    #p-value which is less than 5% so altenate hypothesis correlation exists
    
p_values_df=pd.DataFrame({'Column Name':data_object.columns,'p_value':p_values_list})

p_values_df['p_value'].sort_values(ascending=False)

28     5.001538e-01
32     4.980265e-01
21     4.257982e-01
13     8.561657e-02
16     7.253662e-03
4      1.439614e-03
3      3.093951e-04
27     1.914769e-04
50     1.883395e-05
38     3.911500e-06
8      2.355764e-07
11     3.859509e-12
41     2.888727e-13
49     1.077721e-14
9      1.758909e-15
7      1.434114e-17
26     4.377856e-18
31     1.165152e-18
22     1.210308e-19
25     8.288708e-22
29     7.435297e-22
24     2.775633e-23
36     2.146644e-23
6      1.294722e-23
48     1.018901e-23
19     2.778742e-24
10     2.319780e-24
51     5.747724e-25
30     1.869503e-28
5      1.598215e-28
35     6.692324e-30
44     1.256128e-33
23     1.783875e-35
46     2.917532e-36
47     7.997531e-37
45     1.607312e-37
14     9.852885e-40
2      1.655279e-43
37     1.264234e-44
20     5.262741e-47
42     1.648687e-49
43     3.485656e-52
12     1.027742e-53
18     2.506417e-54
40     1.226526e-55
15     8.543230e-67
17     6.108561e-92
34    2.679675e-100
0     6.699477e-137
33    5.240161e-175


In [125]:
from xgboost import XGRegressor

params={'min_child_weight':[4,5],'gamma':[i/10.0 for i in range(3,6)],'subsample':[i/10.0 for i in range(0,11)],
       'colsample_bytree':
       }

array([[1.99478469e+00, 2.00521531e+00],
       [3.11186412e+02, 3.12813588e+02],
       [5.93947142e+02, 5.97052858e+02],
       [8.54266545e+02, 8.58733455e+02],
       [1.30359180e+03, 1.31040820e+03],
       [2.31195546e+03, 2.32404454e+03],
       [2.53138177e+03, 2.54461823e+03],
       [7.91580436e+03, 7.95719564e+03],
       [1.24155399e+04, 1.24804601e+04],
       [4.98696173e-01, 5.01303827e-01],
       [4.98696173e-01, 5.01303827e-01],
       [1.49608852e+00, 1.50391148e+00],
       [4.98696173e-01, 5.01303827e-01],
       [9.97392346e-01, 1.00260765e+00],
       [4.98696173e-01, 5.01303827e-01],
       [4.98696173e-01, 5.01303827e-01],
       [7.48044260e+00, 7.51955740e+00],
       [4.98696173e-01, 5.01303827e-01],
       [4.98696173e+00, 5.01303827e+00],
       [4.98696173e-01, 5.01303827e-01],
       [4.98696173e-01, 5.01303827e-01],
       [2.99217704e+00, 3.00782296e+00],
       [7.48044260e+00, 7.51955740e+00],
       [4.98696173e-01, 5.01303827e-01],
       [4.488265